In [1]:
import glob
import itertools
import numpy as np
import pandas as pd
import plotly.express as px
from pysam import VariantFile

/var/folders/h7/fr7g30_x5x12d68ww_s6w2_m0000gn/T/ipykernel_4527/1751846874.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
filepath = '/Users/melloo21/Desktop/NUS Items/CS4220/data/test'
# mutec = "real1-mutect2.vcf"
# freebayes = "real1-freebayes.vcf"
# vardict = "real1-vardict.vcf"
# varscan = "real1-varscan.vcf"

In [3]:
files = glob.glob('/Users/melloo21/Desktop/NUS Items/CS4220/data/real1/*.bed',
                   recursive = True) 

for file in files:
    real1_truth_bed = pd.read_csv(file,sep='\t', comment='t', header=None)
header = ["chrom","posStart","posEnd"]
real1_truth_bed.columns = header[:len(real1_truth_bed.columns)]

In [4]:
display(real1_truth_bed.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1319 entries, 0 to 1318
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   chrom     1319 non-null   object
 1   posStart  1319 non-null   int64 
 2   posEnd    1319 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 31.0+ KB


None

In [5]:
print(f" Length of unique start pos {len(real1_truth_bed.posStart.unique())}")

 Length of unique start pos 1319


In [6]:
real1_df = pd.read_csv("/Users/melloo21/Desktop/NUS Items/CS4220/data/original-real-1-parse.txt",sep='\t')

/var/folders/h7/fr7g30_x5x12d68ww_s6w2_m0000gn/T/ipykernel_4527/1117309104.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  real1_df = pd.read_csv("/Users/melloo21/Desktop/NUS Items/CS4220/data/original-real-1-parse.txt",sep='\t')


In [7]:
# all vcf files
display(real1_df.describe())

,START_POS_REF,END_POS_REF,m2_MQ,f_MQMR,vs_SSC,vs_SPV,vd_SSF,vd_MSI
count,4.932000e+04,4.932000e+04,8719.000000,7537.000000,39668.000000,39668.000000,16076.000000,16076.000000
mean,6.509624e+07,6.509624e+07,54.510235,49.345667,15.221009,0.118016,0.101967,4.169232
std,5.462033e+07,5.462033e+07,7.070806,15.615680,17.291468,0.169660,0.186070,5.324648
min,3.000000e+00,3.000000e+00,10.130000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.151707e+07,2.151707e+07,50.680000,43.193500,8.000000,0.018322,0.000110,1.000000
50%,5.135118e+07,5.135118e+07,57.480000,57.583300,12.000000,0.051464,0.024045,2.000000
75%,9.623363e+07,9.623363e+07,60.000000,60.000000,17.000000,0.151600,0.117655,4.000000
max,2.492405e+08,2.492405e+08,62.650000,70.000000,255.000000,1.000000,1.000000,56.000000


In [8]:
real1_truth_bed.columns

Index(['chrom', 'posStart', 'posEnd'], dtype='object')

In [10]:

real1_df["IS_SNV"] = np.where(real1_df.START_POS_REF.isin(real1_truth_bed.posStart),1,0) 

In [11]:
real1_df[real1_df.IS_SNV == 1].isna().sum()/len(real1_df[real1_df.IS_SNV == 1]) * 100

Chr                  0.000000
START_POS_REF        0.000000
END_POS_REF          0.000000
REF                  0.000000
ALT                  0.000000
REF_MFVdVs           0.000000
ALT_MFVdVs           0.000000
Sample_Name          0.000000
FILTER_Mutect2       0.000000
FILTER_Freebayes     0.000000
FILTER_Vardict       0.000000
FILTER_Varscan       0.000000
m2_MQ                0.863422
f_MQMR              10.204082
vs_SSC               6.907378
vs_SPV               6.907378
vd_SSF               8.241758
vd_MSI               8.241758
IS_SNV               0.000000
dtype: float64

In [13]:
# fig = px.scatter(real1_df, x=real1_df.index, y="vs_SSC", color="IS_SNV")
fig = px.histogram(real1_df, x="m2_MQ", color="IS_SNV")
fig.show()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [16]:
real1_df.isna().sum()

Chr                     0
START_POS_REF           0
END_POS_REF             0
REF                     0
ALT                     0
REF_MFVdVs              0
ALT_MFVdVs              0
Sample_Name             0
FILTER_Mutect2          0
FILTER_Freebayes        0
FILTER_Vardict          0
FILTER_Varscan          0
m2_MQ               40601
f_MQMR              41783
vs_SSC               9652
vs_SPV               9652
vd_SSF              33244
vd_MSI              33244
dtype: int64

In [8]:
all_df = real1_truth_bed.merge(real1_df, left_on='posStart', right_on='START_POS_REF')

In [9]:
display(all_df)

,chrom,posStart,posEnd,Chr,START_POS_REF,END_POS_REF,REF,ALT,REF_MFVdVs,ALT_MFVdVs,...,FILTER_Mutect2,FILTER_Freebayes,FILTER_Vardict,FILTER_Varscan,m2_MQ,f_MQMR,vs_SSC,vs_SPV,vd_SSF,vd_MSI
0,1,2180985,2180985,1,2180985,2180985,A,G,A/A/A/A/,G/G/G/G/,...,True,True,True,False,60.00,60.0000,170.0,8.416100e-18,0.00000,2.0
1,1,5035185,5035185,1,5035185,5035185,C,T,C/C/C/C/,T/T/T/T/,...,True,True,True,True,59.77,60.0000,45.0,2.685800e-05,0.00000,1.0
2,1,8881322,8881322,1,8881322,8881322,G,A,G/G/G/G/,A/A/A/A/,...,True,True,True,False,60.00,59.8525,39.0,1.061600e-04,0.00007,3.0
3,1,8929624,8929624,1,8929624,8929624,A,G,A/A/A/A/,G/G/G/G/,...,True,True,True,False,59.58,60.0000,34.0,3.702700e-04,0.00004,5.0
4,1,9196716,9196716,1,9196716,9196716,C,T,C/NA/C/C/,T/NA/T/T/,...,True,False,True,True,60.00,NaN,16.0,2.043900e-02,0.03697,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269,X,141680928,141680928,X,141680928,141680928,C,A,C/C/C/C/,A/A/A/A/,...,True,True,True,True,60.00,60.0000,10.0,8.461500e-02,0.08730,1.0
1270,X,142275497,142275497,X,142275497,142275497,T,C,T/T/T/T/,C/C/C/C/,...,True,True,True,False,60.00,60.0000,73.0,4.030200e-08,0.00000,2.0
1271,X,144737796,144737796,X,144737796,144737796,C,G,C/C/C/C/,G/G/G/G/,...,True,True,True,True,60.00,60.0000,18.0,1.380300e-02,0.01672,1.0
1272,Y,15154839,15154839,Y,15154839,15154839,G,C,G/G/G/G/,C/C/C/C/,...,True,True,True,False,60.00,60.0000,85.0,2.585400e-09,0.00000,6.0


In [18]:
all_df.isna().sum()

chrom                 0
posStart              0
posEnd                0
Chr                   0
START_POS_REF         0
END_POS_REF           0
REF                   0
ALT                   0
REF_MFVdVs            0
ALT_MFVdVs            0
Sample_Name           0
FILTER_Mutect2        0
FILTER_Freebayes      0
FILTER_Vardict        0
FILTER_Varscan        0
m2_MQ                11
f_MQMR              130
vs_SSC               88
vs_SPV               88
vd_SSF              105
vd_MSI              105
dtype: int64

In [56]:
pd.set_option('display.max_rows', 500)
real1_truth_bed[~real1_truth_bed.isin(list(all_df.posStart))].dropna().reset_index()

,index,chrom,posStart,posEnd
0,47,1,105553679.0,105553679.0
1,87,1,236070218.0,236070218.0
2,107,10,27264403.0,27264403.0
3,109,10,29381097.0,29381097.0
4,111,10,37702476.0,37702476.0
5,116,10,55821614.0,55821614.0
6,126,10,75481547.0,75481547.0
7,137,10,100974667.0,100974667.0
8,169,11,38267894.0,38267894.0
9,184,11,68115417.0,68115417.0


In [55]:
mutec = "mutect2.vcf"
freebayes = "freebayes.vcf"
vardict = "vardict.vcf"
varscan = "varscan.vcf"

In [42]:
# open the file
mutec_vcf = VariantFile(f"{filepath}/{mutec}")
# fetch the variants
all_variants = mutec_vcf.fetch()
# only look at 3 records
variants = itertools.islice(all_variants, 121)

In [56]:
# Open all files
all_var_files = {
    "mutec": VariantFile(f"{filepath}/{mutec}"),
    "freebayes": VariantFile(f"{filepath}/{freebayes}"),
    "vardict": VariantFile(f"{filepath}/{vardict}"),
    "varscan": VariantFile(f"{filepath}/{varscan}")
}
# mutec_vcf = VariantFile(f"{filepath}/{mutec}")
# freebayes_vcf = VariantFile(f"{filepath}/{freebayes}")
# vardict_vcf = VariantFile(f"{filepath}/{vardict}")
# varscan_vcf = VariantFile(f"{filepath}/{varscan}")

In [57]:
all_algos = {
    "mutec": (list(),),
    "freebayes": (list(),),
    "vardict": (list(),),
    "varscan": (list(),),
}

for keys in all_algos:
    for record in all_var_files[keys].fetch():
        all_algos[keys][0].append(record.pos)
    all_algos[keys] += (len(all_algos[keys][0]),)

In [58]:
# Print all the lens of each file
for keys, item in all_algos.items():
    print(f"number of positions in {keys} :: {item[1]}")

number of positions in mutec :: 99821
number of positions in freebayes :: 199824
number of positions in vardict :: 199829
number of positions in varscan :: 199874


In [21]:
# Other parameters in the dataset
# 'alleles',
#  'alleles_variant_types',
#  'alts',
#  'chrom',
#  'contig',
#  'copy',
#  'filter',
#  'format',
#  'header',
#  'id',
#  'info',
#  'pos',
#  'qual',
#  'ref',
#  'rid',
#  'rlen',
#  'samples',
#  'start',
#  'stop',
#  'translate'
print(variant.alleles)

('T', 'G')


In [20]:
dir(variant)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'alleles',
 'alleles_variant_types',
 'alts',
 'chrom',
 'contig',
 'copy',
 'filter',
 'format',
 'header',
 'id',
 'info',
 'pos',
 'qual',
 'ref',
 'rid',
 'rlen',
 'samples',
 'start',
 'stop',
 'translate']